<a href="https://colab.research.google.com/github/bmill42/musical-structure/blob/main/Simple_harmonic_Markov_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
!pip install music21 --quiet
!pip install midi_player --quiet

In [ ]:
import random
import music21
from midi_player import MIDIPlayer
from midi_player.stylers import basic, cifka_advanced
from fractions import Fraction

In [ ]:
octave_map = { '-': 48, '=': 60, '+': 72 }

scale_deg_map = { '1': 0, '2': 2, '3': 4, '4': 5, '5': 7, '6': 9, '7': 11 }

def scale_degree_to_num(deg):
    return octave_map[deg[0]] + scale_deg_map[deg[1]]

In [ ]:
def tune_to_string(tune, dur=1, wait=1, rhythms=None):
    pitches = tune.split(' ')
    if rhythms is None:
        return ' '.join(['n_{}_{} w_{}.0'.format(str(scale_degree_to_num(p)), dur, wait) for p in pitches])
    else:
        pitch_rhythms = zip(pitches, rhythms)
        return ' '.join(['n_{}_{} w_{}'.format(str(scale_degree_to_num(pr[0])), pr[1], pr[1]) for pr in pitch_rhythms])

def chords_to_string(progression, dur=1, wait=1):
    chords = progression.split(' ')
    output = ''
    for c in chords:
        pitches = chord_scale_degrees[c]
        output += ' '.join(['n_{}_{}'.format(str(scale_degree_to_num(p)), dur) for p in pitches])
        output += ' w_{} '.format(wait)
    return output

def string_to_midi(tune, dur=1, wait=1, rhythms=None):
    s = tune_to_string(tune, dur, wait, rhythms)
    stream = music21.stream.Stream()
    time = 1
    for i in s.split():
        if i.startswith('n'):
            note, duration = i.lstrip('n_').split('_')
            n = music21.note.Note(int(note))
            n.duration.quarterLength = float(duration)
            stream.insert(time, n)
        elif i.startswith('w'):
            time += float(Fraction(i.lstrip('w_')))
    return stream

def chords_to_midi(prog, dur=2, wait=2):
    s = chords_to_string(prog, dur, wait)
    stream = music21.stream.Stream()
    time = 1
    for i in s.split():
        if i.startswith('n'):
            note, duration = i.lstrip('n_').split('_')
            n = music21.note.Note(int(note))
            n.duration.quarterLength = float(duration)
            stream.insert(time, n)
        elif i.startswith('w'):
            time += float(Fraction(i.lstrip('w_')))
    return stream

def play_midi(tune, dur=1, wait=1, rhythms=None):
    midi = string_to_midi(tune, dur, wait, rhythms)
    midi.write('midi', 'generated.midi')
    return MIDIPlayer('generated.midi', 120, styler=cifka_advanced, title='My Player', width='50%')

def play_midi_chords(prog, dur=2, wait=2):
    midi = chords_to_midi(prog, dur, wait)
    midi.write('midi', 'generated.midi')
    return MIDIPlayer('generated.midi', 120, styler=cifka_advanced, title='My Player', width='50%')

In [ ]:
def next_state(model, cur_state):
    return random.choices(population=list(model[cur_state].keys()), weights=list(model[cur_state].values()), k=1)[0]

def generate(model):
    output = ''
    token = next_state(model, 'START')

    while token != 'END':
        output += token + ' '
        token = next_state(model, token)

    return output.strip()

# Exercise: Building a harmonic model

Just as we can build a Markov model of melodies, rhythms, and intervals, we can also model harmonic progressions. The tokens are just harmonies, so a basic sequence might look like:

```
I IV V I
```

Since chords contain multiple notes, we need an extra step to translate chord symbols into specific pitches, so we'll end up with both a `chord_model` and a mapping of `chord_scale_degrees`.

The first part of the assignment is to fill out the models below with reasonable values.

## A basic harmonic model

First we need to model basic chord transitions. The model is currently very simple and incomplete: I always moves to IV, which always moves to V, which always moves to I. *There is no `'START'` or `'END'` state, so the model can't get started, and if it did, it would generate infinite progressions.*

**Flesh out the model to represent reasonable harmonic progressions, and make sure to incorporate start and end tokens.** The model should include at least 5 different chords, but the selection of chords and their transition probabilities can be drawn from any style of your choice---classical/common practice harmony, rock, pop, etc.

In [ ]:
chord_model = { # Fill this out
    'I': {
            'IV': 1
    },
    'IV': {
            'V': 1
    },
    'V': {
            'I': 1
    }
}

**You'll also need to define the scale degree mappings for the harmonies in your model.** The mappings consist of a dictionary with each chord symbol as keys, and lists of scale degrees as values.

The scale degrees should be represented as numbers 1-7 along with octave designations using -/=/+, as in the melodic model. For example, middle C is `'=1'` and the G below that is `'-5'`.

Note that this means you can choose the exact voicings for your harmonies, including inversions! Also note that in this simple model, only diatonic notes are allowed, no accidentals.

In [ ]:
chord_scale_degrees = { # Fill this out, including entries for all harmonies in the chord_model
    'I': ['-1', '-3', '-5'], # Feel free to replace this with a better voicing
    'IV': [],
    'V': []
}

You can generate a new progression and listen to it here:

In [ ]:
play_midi_chords(generate(chord_model))

## Building a model from a corpus

Finally, you will build another version of the same kind of model, but this time using transition probabilities from a corpus of actual songs.

**Select at least 5 songs/musical works and use the transition probabilities for their harmonies to define this model.**

You may select the pieces for your corpus however you like: songs that are in a similar style by different artists, songs from the same artist, etc. You can either transcribe songs you already know, refer to published transciptions, or look up the chords for songs online (but do try to verify any online transcriptions before using them)

*Depending on the songs you choose, you may also need to create a new `chord_scale_degrees` mapping to map additional chords or new voicings.* If necessary, uncomment the block of code below and fill it out. Just don't change the name of this variable, as the utility functions for the MIDI player depend on it.

In [ ]:
corpus_chord_model = { # Fill this out
    'I': {
            'IV': 1
    },
    'IV': {
            'V': 1
    },
    'V': {
            'I': 1
    }
}

# If necessary, you can uncomment this and create a new mapping of scale degrees
'''chord_scale_degrees = {
    'I': ['-1', '-3', '-5'],
    'IV': [],
    'V': []
}'''

# Writeup

**You should include a very brief writeup explaining:**

*   What style of music you chose to model in the basic harmonic model
*   Which specific songs you included in the corpus for the corpus model
*   What are the strengths and weaknesses of your models? To what extent are the weaknesses a result of your own choices for the corpus and/or transition probabilities, and to what extent are they inherent to the Markov model itself?

You can either write this up in a text cell below this one or as a separate document